## Data

* OLID
    * Only top-level label (i.e. whether message is offensive or not)
    * Subset of complete dataset -> same $n$ & class distribution as HASOC 
* HASOC
* Available at [link](https://canvas.vu.nl/courses/63973/files/5284148?wrap=1)

In [1]:
from pathlib import Path

PATH_DATA = Path().resolve() / 'data'
PATH_DATA.mkdir(exist_ok=True)

Unzip `Subjectivity_mining_assignment_3_4_data.zip` in data folder

In [2]:
PATH_DATA = PATH_DATA / 'Subjectivity_mining_assignment_3_4_data'

# BERT-based model

In [8]:
# !pip install simpletransformers

In [7]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# Optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs=1

model = ClassificationModel(
    "bert", "diptanu/fBERT", args=model_args
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/604 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at diptanu/fBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at diptanu/fBERT and are newly ini

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
import pandas as pd

df_train_olid_small = pd.read_csv(PATH_DATA / 'olid-train-small.csv')
df_train_hasoc = pd.read_csv(PATH_DATA / 'hasoc-train.csv')
df_test_olid = pd.read_csv(PATH_DATA / 'olid-test.csv')

In [13]:
model.train_model(pd.concat([df_train_olid_small, df_train_hasoc]))

  0%|          | 0/11704 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1463 [00:00<?, ?it/s]

(1463, 0.5643336034684824)

In [14]:
result, model_outputs, wrong_predictions = model.eval_model(df_test_olid)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [16]:
from sklearn.metrics import classification_report
import numpy as np

print(classification_report(
    y_true=df_test_olid['labels'],
    y_pred=np.argmax(model_outputs, axis=1)
))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90       620
           1       0.82      0.58      0.68       240

    accuracy                           0.85       860
   macro avg       0.84      0.77      0.79       860
weighted avg       0.85      0.85      0.84       860



Compare to only OLID training data:

```
              precision    recall  f1-score   support

           0       0.87      0.90      0.89       620
           1       0.72      0.65      0.68       240

    accuracy                           0.83       860
   macro avg       0.80      0.78      0.79       860
weighted avg       0.83      0.83      0.83       860
```